In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Taxi_analysis_2024") \
    .config("spark.ui.port", "4041") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.jars", r"C:\Program Files\PostgreSQL\17\jdbc\postgresql-42.7.5.jar") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.hadoop.io.native.lib.available", "false")


In [2]:
import os

directory = r"F:\data project\Taxi_track_2024\data\2024"
data={}
for root, _, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        file_name=file_path.split("\\")[5].split("_")[2].split(".")[0]
        data[file_name]=spark.read.parquet(file_path)
    

In [3]:
lookups={}
directory = r"F:\data project\Taxi_track_2024\data\lookups"
for root, _, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        file_name=file_path.split("\\")[5].split(".")[0]
        lookups[file_name]=spark.read.csv(file_path,header=True,inferSchema=True)
   


In [4]:
from pyspark.sql.functions import col,split, year, month, dayofmonth, hour, minute, when, date_format,unix_timestamp,count,round,concat_ws,lit\
,explode, sequence, to_date, expr, year, month, dayofmonth, date_format

In [31]:
 
c=0
for i in data.keys():
        df=data[i].withColumn('tpep_pickup_datetime',col('tpep_pickup_datetime').cast('timestamp'))\
                        .withColumn('tpep_dropoff_datetime',col('tpep_dropoff_datetime').cast('timestamp'))\
                        .withColumn("pickup_date", split(col("tpep_pickup_datetime")," ")[0])\
                        .withColumn("pickup_time", split(col("tpep_pickup_datetime")," ")[1])\
                        .withColumn("dropoff_date", split(col("tpep_dropoff_datetime")," ")[0])\
                        .withColumn("dropoff_time", split(col("tpep_dropoff_datetime")," ")[1])\
                        .withColumn("pickup_date_key", date_format(col("pickup_date"), "yyyyMMdd").cast("int"))\
                        .withColumn("dropoff_date_key", date_format(col("dropoff_date"), "yyyyMMdd").cast("int"))\
                        .withColumn("duration",round((unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60,2))\
                        .withColumn("pickup_hour", hour("tpep_pickup_datetime")) \
                        .withColumn("pickup_time_of_day", 
                                when(col("pickup_hour").between(0, 5), "Morning")
                                .when(col("pickup_hour").between(6, 11), "Afternoon")
                                .when(col("pickup_hour").between(12, 17), "Evening")
                                .when(col("pickup_hour").between(18, 23), "Night")
                                )\
                        .withColumn("dropoff_hour", hour("tpep_dropoff_datetime")) \
                        .withColumn("dropoff_time_of_day", 
                                when(col("dropoff_hour").between(0, 5), "Morning")
                                .when(col("dropoff_hour").between(6, 11), "Afternoon")
                                .when(col("dropoff_hour").between(12, 17), "Evening")
                                .when(col("dropoff_hour").between(18, 23), "Night")
                                )
        if c==0:
                Taxi_facts=df
                c=1
                continue
        
        Taxi_facts=Taxi_facts.union(df)


Taxi_facts=Taxi_facts.select(
               
                col('VendorID'),
                col('pickup_date_key'),
                col('pickup_time'),
                col('dropoff_date_key'),
                col('dropoff_time'),
                col('trip_distance').alias("trip_distance_in_mile"),
                col('RatecodeID'),
                col('store_and_fwd_flag'),
                col('PULocationID'),
                col('DOLocationID'),
                col('payment_type'),               
                col('duration'),
                col('passenger_count'),
                col('fare_amount'),
                col('extra'),
                col('mta_tax'),
                col('tip_amount'),
                col('improvement_surcharge'),
                col('total_amount'),
                col('congestion_surcharge'),
                col('Airport_fee')
            )

Taxi_facts=Taxi_facts.fillna({"passenger_count": 0})\
                    .fillna({"RatecodeID": 0})\
                    .fillna({"store_and_fwd_flag": 'None'})\
                   .fillna({"congestion_surcharge": 0})\
                   .fillna({"Airport_fee": 0})\
                  



In [16]:
Taxi_facts.select('store_and_fwd_flag').distinct().show()

+------------------+
|store_and_fwd_flag|
+------------------+
|                 Y|
|                 N|
|              None|
+------------------+



In [6]:


 
# Define start and end dates
start_date = "2024-01-01"
end_date = "2024-12-31"

# Create a DataFrame with a sequence of dates
date_dim = spark.createDataFrame([(start_date, end_date)], ["start_date", "end_date"]) \
       .withColumn("start_date", to_date(col("start_date"))) \
       .withColumn("end_date", to_date(col("end_date"))) \
       .withColumn("date", explode(sequence(col("start_date"), col("end_date"), expr("INTERVAL 1 DAY"))))

# Add date attributes
date_dim = date_dim.withColumn("date_key", date_format(col("date"), "yyyyMMdd").cast("int"))\
       .withColumn("year", year(col("date"))) \
       .withColumn("month", month(col("date"))) \
       .withColumn("day", dayofmonth(col("date"))) \
       .withColumn("day_of_week", date_format(col("date"), "EEEE")) \
       


In [ ]:
from pyspark.sql import Row

In [ ]:
new_row = Row(0, "unkown")
new_df = spark.createDataFrame([new_row], lookups['payment_type_lookup'].schema)
lookups['payment_type_lookup']=lookups['payment_type_lookup'].union(new_df)

In [19]:

new_row = Row("None", "unkown")
new_df = spark.createDataFrame([new_row], lookups['Store_and_fwd_flag'].schema)
lookups['Store_and_fwd_flag']=lookups['Store_and_fwd_flag'].union(new_df)

In [20]:

new_row = Row(0, "unkown")
new_df = spark.createDataFrame([new_row], lookups['rate_lookup'].schema)
lookups['rate_lookup']=lookups['rate_lookup'].union(new_df)

In [21]:
Taxi_facts.show()

+--------+---------------+-----------+----------------+------------+-------------+----------+------------------+------------+------------+------------+--------+---------------+-------------+-----------+-----+-------+----------+---------------------+------------+--------------------+-----------+
|VendorID|pickup_date_key|pickup_time|dropoff_date_key|dropoff_time|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|duration|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+---------------+-----------+----------------+------------+-------------+----------+------------------+------------+------------+------------+--------+---------------+-------------+-----------+-----+-------+----------+---------------------+------------+--------------------+-----------+
|       2|       20240101|   00:57:55|        20240101|    01:17:43|         1.72|         1|                 N|

In [24]:
lookups['Store_and_fwd_flag'].show()

+----+--------------------+
|flag|           flag_name|
+----+--------------------+
|   Y|store and forward...|
|   N|not a store and f...|
|None|              unkown|
+----+--------------------+



In [25]:
lookups['payment_type_lookup'].show()

+----------+------------+
|payment_id|payment_name|
+----------+------------+
|         1| Credit card|
|         2|        Cash|
|         3|   No charge|
|         4|     Dispute|
|         5|     Unknown|
|         6| Voided trip|
|         0|      unkown|
+----------+------------+



In [26]:
lookups['rate_lookup'].show()

+-------+--------------------+
|rate_id|           rate_name|
+-------+--------------------+
|      1|       Standard rate|
|      2|                 JFK|
|      3|              Newark|
|      4|Nassau or Westche...|
|      5|     Negotiated fare|
|      6|          Group ride|
|      0|              unkown|
+-------+--------------------+



In [2]:
jdbc_url = "jdbc:postgresql://localhost:5432/Taxi_2024_warhouse"
db_properties = {
    "user": "postgres",
    "password": 123,
    "driver": "org.postgresql.Driver"
}

In [2]:
def load_into_DataWarhouse(df,jdbc_url,db_properties,table_name):
    df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", f"public.{table_name}") \
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .mode("overwrite") \
    .save()


In [ ]:
load_into_DataWarhouse(lookups['Store_and_fwd_flag'],jdbc_url,db_properties,'Store_and_fwd_flag_dim')
load_into_DataWarhouse(lookups['payment_type_lookup'],jdbc_url,db_properties,'payment_type_dim')
load_into_DataWarhouse(lookups['rate_lookup'],jdbc_url,db_properties,'rate_dim')
load_into_DataWarhouse(lookups['taxi_zone_lookup'],jdbc_url,db_properties,'taxi_zone_dim')
load_into_DataWarhouse(lookups['vendor_lookup'],jdbc_url,db_properties,'vendor_dim')
load_into_DataWarhouse(date_dim,jdbc_url,db_properties,'date_dim')

In [32]:
load_into_DataWarhouse(Taxi_facts,jdbc_url,db_properties,'Taxi_facts')

In [3]:
def reading_data(table_name,jdbc_url,db_properties,spark):
    df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", f"public.{table_name}")\
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .load()

    return df